## ML Zoom Camp 2022
### Week 8 assignment

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon). 

The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and testing.

I have chosen the first 20% of the pictures in the train folder to be in the validation folder.

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from tensorflow import keras
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers

In [3]:
def make_model(learning_rate=0.1, size_inner=100):
    inputs = keras.Input(shape=(150, 150, 3))
    conv_out = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
    pool_out = layers.MaxPooling2D(pool_size=(2, 2))(conv_out)
    vectors = layers.Flatten()(pool_out)
    inner = layers.Dense(size_inner, activation='relu')(vectors)
    outputs = layers.Dense(1, activation='sigmoid')(inner)
    model = keras.Model(inputs, outputs)
    
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [4]:
lr = 0.002
dense = 64
model = make_model(learning_rate=lr, size_inner=dense)

### Question 1

Since we have a binary classification problem, what is the best loss function for us?
- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set from_logits=True

Answer: should be BinaryCrossentropy.


### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

As we can see from the Total params value - 11215873

### Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    '..\\data\\train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = datagen.flow_from_directory(
    '..\\data\\train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1594 images belonging to 2 classes.
Found 1594 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'dino': 0, 'dragon': 1}

In [9]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
80/80 [==============================] - 37s 450ms/step - loss: 0.5919 - accuracy: 0.6882 - val_loss: 0.4494 - val_accuracy: 0.8476
Epoch 2/10
80/80 [==============================] - 34s 429ms/step - loss: 0.4301 - accuracy: 0.8256 - val_loss: 0.4329 - val_accuracy: 0.8030
Epoch 3/10
80/80 [==============================] - 35s 438ms/step - loss: 0.3538 - accuracy: 0.8551 - val_loss: 0.3357 - val_accuracy: 0.8670
Epoch 4/10
80/80 [==============================] - 35s 436ms/step - loss: 0.3035 - accuracy: 0.8864 - val_loss: 0.3324 - val_accuracy: 0.8626
Epoch 5/10
80/80 [==============================] - 34s 432ms/step - loss: 0.2760 - accuracy: 0.8921 - val_loss: 0.2312 - val_accuracy: 0.9216
Epoch 6/10
80/80 [==============================] - 34s 429ms/step - loss: 0.2506 - accuracy: 0.8984 - val_loss: 0.2213 - val_accuracy: 0.9247
Epoch 7/10
80/80 [==============================] - 35s 434ms/step - loss: 0.2218 - accuracy: 0.9172 - val_loss: 0.1711 - val_accuracy: 0.9448

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [10]:
np.median(history.history['val_accuracy'])

0.9231492877006531

In [11]:
np.median(history.history['accuracy'])

0.8952321410179138


### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33



In [12]:
np.std(history.history['val_loss'])

0.11683011115477356

In [13]:
np.std(history.history['loss'])

0.12987681979177018


### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`


In [14]:
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_generator_aug = train_datagen_aug.flow_from_directory(
    '..\\data\\train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1594 images belonging to 2 classes.



### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97


In [ ]:
history_aug = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

In [ ]:
np.average(history_aug.history['val_loss'])

0.08340817093849182


### Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24


In [ ]:
np.average(history_aug.history['val_accuracy'][-5:])

0.995984947681427